In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score 
from sklearn.model_selection import train_test_split
import random

import matplotlib.pyplot as plt
import pickle
from matplotlib.pyplot import figure
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# the data can be downloaded at https://osf.io/tb8fx/
df = pd.read_csv("combined", index_col = 0)

# Helper Functions

In [ ]:
def q_plus(v, alpha):
    n = v.shape[0]
    r = (np.ceil((1-alpha)*(n+1)) - 1).astype(int)
    if r == n:
        return float("inf")
    return np.sort(v)[r]

def q_minus(v, alpha):
    n = v.shape[0]
    r = (np.floor(alpha*(n+1)) - 1).astype(int)
    if r == -1:
        return -float("inf")
    return np.sort(v)[r]

# Split Conformal

In [ ]:
def split_conformal(df_train, df_test, alpha_delta_list, train_ratio=0.6):        
    train_idx = df_train["Electrode"].unique()
    fit_idx, residual_idx = train_test_split(train_idx, train_size=train_ratio)
    
    # df_fit is the data used to fit the models
    df_fit = df_train[df_train.Electrode.isin(fit_idx)]
    # df_fit is the data used to compute the residuals
    df_residual = df_train[df_train.Electrode.isin(residual_idx)]

    lin_model = get_model()
    lin_model.fit(df_fit.drop(columns=["DA", "Electrode"]), df_fit["DA"])
    
    # compute residuals using the remaining training environments
    training_residuals = {}
    for i in residual_idx:
        curr_df = df_residual[df_residual.Electrode == i]
        residual = np.abs(curr_df["DA"] - lin_model.predict(curr_df.drop(columns=["DA", "Electrode"])))
        training_residuals[i] = residual
    
    # compute the residual quantiles
    q_dict = {}
    for alpha, delta in alpha_delta_list:  
        quantile_residuals = []
        for i in residual_idx:
            residual = training_residuals[i]
            quantile_residuals.append(q_plus(residual, alpha))
        quantile_residuals = np.array(quantile_residuals)
        q_dict[(alpha, delta)] = q_plus(quantile_residuals, delta)

    # this stores the total cardinality of constructed confidence sets
    total_card_list = defaultdict(list)
    # this stores the number of total covered samples
    total_covered_list = defaultdict(list)
    # this stores the number of total samples
    total_count_list = defaultdict(list)
    
    test_idx = df_test["Electrode"].unique()
    for test_idx_num in test_idx:
        curr_df = df_test[df_test.Electrode == test_idx_num]
        y_pred = lin_model.predict(curr_df.drop(columns=["DA", "Electrode"]))
        y_true = curr_df["DA"]

        for alpha, delta in alpha_delta_list:
            q = q_dict[(alpha, delta)]
            hi, lo = y_pred + q, y_pred - q
            hi[hi > 2000], hi[hi < 0] = 2000, 0
            lo[lo > 2000], lo[lo < 0] = 2000, 0
            
            # set size
            total_card_list[(alpha, delta)].append(np.sum(hi - lo))
            # ratio of covered samples
            total_covered_list[(alpha, delta)].\
            append(np.where(np.logical_and(y_true>=lo, y_true<=hi))[0].shape[0])
            # number of total samples
            total_count_list[(alpha, delta)].append(y_pred.shape[0])

    results_dict = {}
    for alpha, delta in alpha_delta_list:
        curr_total_covered_list = np.array(total_covered_list[(alpha, delta)])
        curr_total_card_list = np.array(total_card_list[(alpha, delta)])
        curr_total_count_list = np.array(total_count_list[(alpha, delta)])

        # compute empirical delta
        emp_delta = np.mean(curr_total_covered_list >= np.ceil((curr_total_count_list + 1) * (1 - alpha)))
        # compute average set size
        emp_card = np.mean(curr_total_card_list / curr_total_count_list)
        # compute empirical alpha
        is_valid_coverage = (curr_total_covered_list >= (1-alpha)*(1+curr_total_count_list))
        coverage_ratio = curr_total_covered_list / curr_total_count_list
        emp_alpha = np.mean(coverage_ratio[is_valid_coverage])
        
        results_dict[(alpha, delta)] = (emp_delta, emp_alpha, emp_card)
    return results_dict

# Jackknife Conformal

In [ ]:
def jackknife_conformal(df_train, df_test, alpha_delta_list):
    train_idx = df_train["Electrode"].unique()
    
    # train the LOO models
    models = []
    for i in train_idx:
        df_concate = df_train[df_train.Electrode != i]
        LOO_model = get_model()
        LOO_model.fit(df_concate.drop(columns=["DA", "Electrode"]), df_concate["DA"])
        models.append(LOO_model)
    
    # compute the residuals
    residual_quantiles_dict = defaultdict(list)
    for i in train_idx:
        df_curr = df_train[df_train.Electrode == i]
        residual = np.abs(df_curr["DA"] - LOO_model.predict(df_curr.drop(columns=["DA", "Electrode"])))
        for alpha, delta in alpha_delta_list: 
            residual_quantiles_dict[(alpha, delta)].append(q_plus(residual, alpha))

    # compute the residual quantiles
    q_dict = {}
    for alpha, delta in alpha_delta_list:
        q_dict[(alpha, delta)] = q_plus(np.array(residual_quantiles_dict[(alpha, delta)]), delta)


    total_card_list = defaultdict(list)
    total_covered_list = defaultdict(list)
    total_count_list = defaultdict(list)     

    test_idx = df_test["Electrode"].unique()    
    for test_idx_num in test_idx:
        curr_df = df_test[df_test.Electrode == test_idx_num]
        y_true = curr_df["DA"]
        # stores the predictions by the trained LOO models
        pred_list = []

        for i in range(len(models)):
            curr_model = models[i]
            curr_pred = curr_model.predict(curr_df.drop(columns=["DA", "Electrode"]))
            pred_list.append(curr_pred)

        pred_min = np.min(pred_list, axis=0)
        pred_max = np.max(pred_list, axis=0)

        for alpha, delta in alpha_delta_list:
            q = q_dict[(alpha, delta)]
            lo, hi = pred_min - q, pred_max + q
            hi[hi > 2000], hi[hi < 0] = 2000, 0
            lo[lo > 2000], lo[lo < 0] = 2000, 0

            # set size
            total_card_list[(alpha, delta)].append(np.sum(hi - lo))
            # ratio of covered samples
            total_covered_list[(alpha, delta)].\
            append(np.where(np.logical_and(y_true>=lo, y_true<=hi))[0].shape[0])
            # number of total samples
            total_count_list[(alpha, delta)].append(y_pred.shape[0])

    results_dict = {}
    for alpha, delta in alpha_delta_list:
        curr_total_covered_list = np.array(total_covered_list[(alpha, delta)])
        curr_total_card_list = np.array(total_card_list[(alpha, delta)])
        curr_total_count_list = np.array(total_count_list[(alpha, delta)])

        # compute empirical delta
        emp_delta = np.mean(curr_total_covered_list >= np.ceil((curr_total_count_list + 1) * (1 - alpha)))
        # compute average set size
        emp_card = np.mean(curr_total_card_list / curr_total_count_list)
        # compute empirical alpha
        is_valid_coverage = (curr_total_covered_list >= (1-alpha)*(1+curr_total_count_list))
        coverage_ratio = curr_total_covered_list / curr_total_count_list
        emp_alpha = np.mean(coverage_ratio[is_valid_coverage])
        
        results_dict[(alpha, delta)] = (emp_delta, emp_alpha, emp_card)
    return results_dict

# Experiments

In [ ]:
alpha_delta_list = []
for alpha in np.linspace(0.02, 0.50, 30):
    for delta in np.linspace(0.02, 0.50, 30):
        alpha_delta_list.append((alpha, delta))

In [ ]:
def get_model():
    return RidgeCV()

In [ ]:
def get_results(df, alpha_delta_list, num_experiments=20):
    split_results_list = []
    jackknife_results_list = []
    
    for j in range(num_experiments):
        print(f"Processing iteration {j}")
        
        train_indices, test_indices = train_test_split(df["Electrode"].unique(), train_size=0.34)
        df_train = df[df.Electrode.isin(train_indices)].drop(columns = ["Channel", "pH"])
        df_test = df[~df.Electrode.isin(train_indices)].drop(columns = ["Channel", "pH"])

        # split conformal
        split_results_list.append(split_conformal(df_train, df_test, alpha_delta_list))

        # jackknife conformal
        jackknife_results_list.append(jackknife_conformal(df_train, df_test, alpha_delta_list))
        
    return split_results_list, jackknife_results_list

In [ ]:
# Each element in split_results_list and jackknife_results_list is a dictionary, 
# where the keys are (alpha, delta) pairs, and the values
# are tuples of (empirical delta, empirical alpha, average set size)

split_results_list, jackknife_results_list = get_results(df, alpha_delta_list, 20)